In [ ]:
dataset_path = './datasets/'

In [ ]:
import datasets

embeddings = datasets.load_from_disk(dataset_path + "embeddings")['embeddings']
labels = datasets.load_from_disk(dataset_path + "labels")['labels']

train = datasets.load_from_disk(dataset_path + "train_dataset")
test = datasets.load_from_disk(dataset_path + "test_dataset")
validate = datasets.load_from_disk(dataset_path + "validate_dataset")

# Transformer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device:', device)

### Positional Encoding

Encoder

In [ ]:
# Relative position encoding

class EncoderPositionalEncoding(nn.Module):
    def __init__(self, d_model):
        super(EncoderPositionalEncoding, self).__init__()

        self.fc1 = nn.Linear(4, d_model)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(d_model, d_model)

    def forward(self, x, pos):
        pos = self.fc1(pos)
        pos = self.relu(pos)
        pos = self.fc2(pos)
        return x + pos

Decoder

In [ ]:
class DecoderPositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(DecoderPositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

### Multihead Attention

Encoder

In [ ]:
class EncoderMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, max_seq_length):
        super(EncoderMultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.max_seq_length = max_seq_length # Maximum sequence length for positional encoding
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

        self.pe = nn.ParameterList([nn.Linear(max_seq_length, self.d_model * max_seq_length)] * 4)

    def scaled_dot_product_attention(self, Q, K, V, pe, mask=None):
        # Calculate attention scores
        attn_scores = (torch.matmul(Q, K.transpose(-2, -1)) + pe) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def split_pe(self, pe):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, tmp = pe.size()
        return pe.view(batch_size, self.num_heads, self.d_k, self.max_seq_length)

    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, pos, mask=None):
        # Apply linear transformations and split heads
        Q = self.W_q(Q)
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        Q = self.split_heads(Q)
        batch_size = Q.shape[0]
        pe = sum(self.split_pe(self.pe[i](pos[:, :, i])) for i in range(4))
        pe = torch.matmul(Q, pe)

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, pe, mask)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

Decoder

In [ ]:
class DecoderMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, max_seq_length):
        super(DecoderMultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.max_seq_length = max_seq_length # Maximum sequence length for positional encoding
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

        self.pe = nn.Parameter(torch.randn(self.d_k, max_seq_length)) # Relative position encoding

    def scaled_dot_product_attention(self, Q, K, V, pe, mask=None):
        # Calculate attention scores
        attn_scores = (torch.matmul(Q, K.transpose(-2, -1)) + pe) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        pe = torch.matmul(Q, self.pe[:, self.max_seq_length - K.shape[2]:])

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, pe, mask)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

### Position-wise FFN

In [ ]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

### Encoder

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, max_seq_length):
        super(EncoderLayer, self).__init__()
        self.self_attn = EncoderMultiHeadAttention(d_model, num_heads, max_seq_length)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, pos, mask):
        attn_output = self.self_attn(x, x, x, pos, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

### Decoder

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, max_seq_length):
        super(DecoderLayer, self).__init__()
        self.self_attn = DecoderMultiHeadAttention(d_model, num_heads, max_seq_length)
        self.cross_attn = DecoderMultiHeadAttention(d_model, num_heads, max_seq_length)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

### Transformer

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_encoder_layers, num_decoder_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_encoder_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_decoder_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, pos, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.encoder_embedding(src))
        tgt_embedded = self.dropout(self.decoder_embedding(tgt))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, pos, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# Training the model

In [ ]:
d_model = 64
num_heads = 4
num_encoder_layers = 4
num_decoder_layers = 4
d_ff = 256
dropout = 0.1
src_vocab_size = len(embeddings)
tgt_vocab_size = len(labels) + 2
max_seq_length = 200
batch_size = 32

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model,
                          num_heads, num_encoder_layers, num_decoder_layers,
                          d_ff, max_seq_length, dropout).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)
optimizer = optim.Adam(transformer.parameters(), lr=0.00001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
transformer.load_state_dict(torch.load('transformer_model.sav'))

In [ ]:
def create_dataset(data, maxsize=-1):
    emb = data["embeddings"]
    pos = data["pos"]
    formula = data["formula"]
    if maxsize == -1:
        maxsize = len(emb)
    tmp_emb, tmp_pos, tmp_formula = [], [], []
    print(f"Starting with {maxsize}")
    for i in range(maxsize):
        try:
            if i % 10000 == 0:
                print("Processed", i, "/", maxsize)
            if len(emb[i]) == 0 or len(pos[i]) == 0 or len(formula[i]) == 0:
                continue
            if len(emb[i]) > max_seq_length or len(formula[i]) > max_seq_length:
                continue
            # Efficient embedding padding
            emb_row = torch.tensor(emb[i], dtype=torch.long).reshape((-1))

            # Efficient min calculation
            pos_i = pos[i]
            pos_arr = torch.tensor(pos_i, dtype=torch.float)
            minx = pos_arr[:, 0].min().item() if len(pos_arr) > 0 else 0
            miny = pos_arr[:, 1].min().item() if len(pos_arr) > 0 else 0
            dak = torch.full((max_seq_length, 4), -1000000, dtype=torch.float)
            for j, x in enumerate(pos_i):
                dak[j] = torch.tensor(
                    [x[0] - minx, x[1] - miny, x[2] - minx, x[3] - miny],
                    dtype=torch.float,
                )

            # Efficient formula padding
            padded_formula = torch.tensor(formula[i], dtype=torch.long)

            tmp_emb.append(emb_row)
            tmp_pos.append(dak)
            tmp_formula.append(padded_formula)
        except Exception as e:
            print(f"Skipped sample {i} due to error: {e}")
            continue
    print("Finished processing")
    tmp_emb[0] = torch.concat(
        [tmp_emb[0], torch.zeros(max_seq_length - len(tmp_emb[0]), dtype=torch.long)]
    )
    tmp_formula = [
        torch.cat(
            [f, torch.tensor([len(labels) + 1] * (max_seq_length - len(f)), dtype=torch.long)]
        )
        for f in tmp_formula
    ]
    emb = torch.nn.utils.rnn.pad_sequence(tmp_emb, batch_first=True, padding_value=0)
    pos = torch.stack(tmp_pos)
    formula = torch.nn.utils.rnn.pad_sequence(tmp_formula, batch_first=True, padding_value=len(labels) + 1)
    return emb, pos, formula

In [ ]:
src_train, pos_train, tgt_train = create_dataset(train)
src_test, pos_test, tgt_test = create_dataset(test)
src_validate, pos_validate, tgt_validate = create_dataset(validate)

In [ ]:
train_loader = data.DataLoader(data.TensorDataset(src_train, pos_train, tgt_train), batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(data.TensorDataset(src_test, pos_test, tgt_test), batch_size=batch_size, shuffle=False)
validate_loader = data.DataLoader(data.TensorDataset(src_validate, pos_validate, tgt_validate), batch_size=batch_size, shuffle=False)

In [ ]:
def calculate_accuracy(outputs, targets, pad_token=0):
    """
    Calculate accuracy for sequence-to-sequence tasks.
    
    Args:
        outputs: Model predictions [batch_size, seq_len, vocab_size]
        targets: Target sequences [batch_size, seq_len]
        pad_token: Token used for padding (default: 0)
    
    Returns:
        accuracy: Percentage of correctly predicted tokens (excluding padding)
    """
    # Get predicted tokens (argmax over vocab dimension)
    predictions = torch.argmax(outputs, dim=-1)  # [batch_size, seq_len]
    
    # Create mask to ignore padding tokens
    mask = (targets != pad_token)
    
    # Calculate correct predictions
    correct = (predictions == targets) & mask
    
    # Calculate accuracy as ratio of correct predictions to total non-padding tokens
    accuracy = correct.sum().float() / mask.sum().float()
    
    return accuracy.item() * 100  # Return as percentage


def calculate_sequence_accuracy(outputs, targets, pad_token=0):
    """
    Calculate sequence-level accuracy (exact match for entire sequences).
    
    Args:
        outputs: Model predictions [batch_size, seq_len, vocab_size]
        targets: Target sequences [batch_size, seq_len]
        pad_token: Token used for padding (default: 0)
    
    Returns:
        sequence_accuracy: Percentage of completely correct sequences
    """
    # Get predicted tokens
    predictions = torch.argmax(outputs, dim=-1)  # [batch_size, seq_len]
    
    # Create mask to ignore padding tokens
    mask = (targets != pad_token)
    
    # Check if predictions match targets for non-padding positions
    matches = (predictions == targets) | ~mask  # True for correct or padding positions
    
    # Check if all tokens in each sequence are correct
    sequence_correct = matches.all(dim=1)  # [batch_size]
    
    # Calculate sequence accuracy
    sequence_accuracy = sequence_correct.float().mean()
    
    return sequence_accuracy.item() * 100  # Return as percentage

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
nltk.download('punkt')

def calculate_bleu(predictions, references):
    """
    Calculate average BLEU score for a batch of predictions and references.
    Args:
        predictions: list of predicted token lists
        references: list of reference token lists
    Returns:
        Average BLEU score (0-100)
    """
    smoothie = SmoothingFunction().method4
    scores = []
    for pred, ref in zip(predictions, references):
        # BLEU expects list of reference lists
        scores.append(sentence_bleu([ref], pred, smoothing_function=smoothie))
    return 100 * sum(scores) / len(scores) if scores else 0

In [ ]:
print('Start training...')

transformer.train()

for epoch in range(100):
    # Training phase
    sum_loss = 0
    total_accuracy = 0
    total_seq_accuracy = 0
    num_batches = 0
    
    for i, (src_data, pos_data, tgt_data) in enumerate(train_loader):
        
        if i % 1000 == 0: print('Processed', i, '/', len(train_loader))
        optimizer.zero_grad()
        src_data = src_data.to(device)
        pos_data = pos_data.to(device)
        tgt_data = tgt_data.to(device) 

        output = transformer(src_data, pos_data, tgt_data[:, :-1])
        
        loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
        sum_loss += loss.item()
        
        # Calculate training accuracy
        with torch.no_grad():
            batch_accuracy = calculate_accuracy(output, tgt_data[:, 1:], pad_token=tgt_vocab_size - 1)
            batch_seq_accuracy = calculate_sequence_accuracy(output, tgt_data[:, 1:], pad_token=tgt_vocab_size - 1)
            total_accuracy += batch_accuracy
            total_seq_accuracy += batch_seq_accuracy
            num_batches += 1

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(transformer.parameters(), max_norm=1.0)
        
        loss.backward()
        optimizer.step()

    # Calculate and print average training metrics
    avg_train_loss = sum_loss / len(train_loader)
    avg_train_accuracy = total_accuracy / num_batches
    avg_train_seq_accuracy = total_seq_accuracy / num_batches
    
    print(f"Epoch: {epoch+1}")
    print(f"  Training Loss: {avg_train_loss:.4f}")
    print(f"  Training Token Accuracy: {avg_train_accuracy:.2f}%")
    print(f"  Training Sequence Accuracy: {avg_train_seq_accuracy:.2f}%")

    # Validation phase
    transformer.eval()
    
    with torch.no_grad():
        sum_val_loss = 0
        total_val_accuracy = 0
        total_val_seq_accuracy = 0

        all_preds = []
        all_refs = []

        for i, (src_data, pos_data, tgt_data) in enumerate(validate_loader):
            if i % 1000 == 0: 
                print('Processed', i, '/', len(validate_loader))
            
            src_data = src_data.to(device)
            pos_data = pos_data.to(device)
            tgt_data = tgt_data.to(device) 

            output = transformer(src_data, pos_data, tgt_data[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
            sum_val_loss += loss.item()
            
            # Calculate validation accuracy
            batch_accuracy = calculate_accuracy(output, tgt_data[:, 1:], pad_token=tgt_vocab_size - 1)
            batch_seq_accuracy = calculate_sequence_accuracy(output, tgt_data[:, 1:], pad_token=tgt_vocab_size - 1)
            total_val_accuracy += batch_accuracy
            total_val_seq_accuracy += batch_seq_accuracy

            # BLEU: decode predictions and references to token lists
            preds = torch.argmax(output, dim=-1).cpu().tolist()
            refs = tgt_data[:, 1:].cpu().tolist()
            for pred, ref in zip(preds, refs):
                # Remove padding tokens (tgt_vocab_size - 1)
                pred_tokens = [labels[x - 1] for x in pred if x != (tgt_vocab_size - 1) and x > 0 and x <= len(labels)]
                ref_tokens = [labels[x - 1] for x in ref if x != (tgt_vocab_size - 1) and x > 0 and x <= len(labels)]
                all_preds.append(pred_tokens)
                all_refs.append(ref_tokens)

        avg_val_loss = sum_val_loss / len(validate_loader)
        avg_val_accuracy = total_val_accuracy / len(validate_loader)
        avg_val_seq_accuracy = total_val_seq_accuracy / len(validate_loader)

        # Calculate BLEU score for the epoch
        bleu_score = calculate_bleu(all_preds, all_refs)
        
        print(f"  Validation Loss: {avg_val_loss:.4f}")
        print(f"  Validation Token Accuracy: {avg_val_accuracy:.2f}%")
        print(f"  Validation Sequence Accuracy: {avg_val_seq_accuracy:.2f}%")
        print(f"  Validation BLEU Score: {bleu_score:.2f}")
        
        print("-" * 50)

    # Switch back to training mode
    transformer.train()
    
    torch.save(transformer.state_dict(), 'transformer' + str(sum_loss / len(train_loader)) + '.sav')

In [ ]:
torch.save(transformer.state_dict(), 'transformer_model.sav')

In [ ]:
def predict(id):
    with torch.no_grad():
        output = transformer(src_data[id:id+1].to(device), pos_data[id:id+1].to(device), tgt_data[id:id+1,:-1].to(device)).to(device)
        output = output[0]
        ans = []
        for i in range(len(output)):
            ans.append(torch.argmax(output[i]))
        ans = torch.tensor(ans)

        print('model output:', end=' ')
        for x in ans:
            if x.item() == len(labels) + 1: continue
            print(labels[x.item() - 1], end=' ')
        print()

        print('answer:', end=' ')
        for x in tgt_data[0]:
            if x.item() == len(labels) + 1: continue
            print(labels[x.item() - 1], end=' ')

In [ ]:
predict(0)